In [1]:
import matplotlib.pyplot as plt
import numpy as np
from gym_unity.envs import UnityEnv

import torch
%matplotlib inline


In [2]:
from ddpg_agent import Agent

ModuleNotFoundError: No module named 'ddpg_agent'

In [ ]:
env = UnityEnv("C:/Users/User/Desktop/Police-Unity.exe",worker_id=0)

In [ ]:
for episode in range(10):
    init = env.reset()
    done = False
    ereward = 0
    while not done:
        observation,reward,done,info = env.step(env.action_space.sample())
        ereward += 1
    
    print(ereward)

In [ ]:
agent = Agent(state_size=env.observation_space.shape[0],
             action_size=env.action_space.shape[0],
             random_seed=2)

In [ ]:
agent.actor_local.load_state_dict(torch.load("easy_actor.pth"))
agent.critic_local.load_state_dict(torch.load("easy_critic.pth"))

In [ ]:
n_ep = 100000
agent.noise.sigma=2

In [ ]:
scores=[]
success=[]
for i in range(1,n_ep+1):
    state = env.reset()
    score = 0
    while True:
        action = agent.act(state)
        next,reward,done,_= env.step(action)
        agent.step(state,action,reward,next,done)
        state=next
        score+=reward
        if done:
            success+=[reward>0]
            break
    scores+=[score]
    agent.noise.sigma*=0.995
    torch.save(agent.actor_local.state_dict(),"hard_actor.pth")
    torch.save(agent.critic_local.state_dict(),"hard_critical.pth")
    
    print(i,np.mean(scores))

In [ ]:
env.close()